In [ ]:
import cv2
import os

# --- Paramètres ---
VIDEO_PATH = "nom_du_film"  # Mettez le chemin de votre fichier vidéo
OUTPUT_FOLDER = "captures_film_2"  # Nom du dossier où sauvegarder les images
SECONDS_INTERVAL = 8  # Sauvegarder une image toutes les 10 secondes
# --------------------

# Crée le dossier de sortie s'il n'existe pas
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Ouvre le fichier vidéo
vidcap = cv2.VideoCapture(VIDEO_PATH)

# Récupère le nombre d'images par seconde (FPS) de la vidéo
fps = vidcap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps * SECONDS_INTERVAL) # Calcule le nombre d'images à sauter

print(f"Vidéo chargée. FPS : {fps}. Capture d'une image toutes les {frame_interval} images.")

count = 0
frame_number = 0
success = True

while success:
    # Lit l'image suivante
    success, image = vidcap.read()
    
    if not success:
        break # Fin de la vidéo

    # Vérifie si c'est le moment de sauvegarder l'image
    if frame_number % frame_interval == 0:
        # Calcule le temps (en secondes) pour le nom du fichier
        time_sec = frame_number / fps
        time_min = int(time_sec // 60)
        time_s = int(time_sec % 60)
        
        # Nom du fichier (ex: capture_min_02_sec_30.jpg)
        filename = f"capture_min_{time_min:02d}_sec_{time_s:02d}.jpg"
        filepath = os.path.join(OUTPUT_FOLDER, filename)
        
        # Sauvegarde l'image
        cv2.imwrite(filepath, image)
        print(f"Image sauvegardée : {filepath}")
        count += 1
        
    frame_number += 1

vidcap.release()
print(f"Terminé. {count} images ont été extraites.")

In [ ]:
from moviepy import VideoFileClip

VIDEO_PATH = "nom_du_film.mkv"
AUDIO_OUTPUT = "audio_du_film.wav" # Le format .wav est idéal pour la transcription

print("Démarrage de l'extraction audio...")

try:
    # Charge le fichier vidéo
    video = VideoFileClip(VIDEO_PATH)
    
    # Extrait l'audio et le sauvegarde
    video.audio.write_audiofile(AUDIO_OUTPUT)
    
    video.close()
    print(f"Extraction terminée. Fichier audio sauvegardé : {AUDIO_OUTPUT}")

except Exception as e:
    print(f"Une erreur est survenue : {e}")
    print("Assurez-vous que le fichier vidéo existe et que FFMPEG est installé.")

In [ ]:
import whisper
import time

AUDIO_FILE = "audio_du_film.wav"
TEXT_OUTPUT = "dialogues_du_film.txt"

# "tiny", "base", "small", "medium", "large"
# "base" est un bon compromis vitesse/précision.
# "medium" ou "large" sont plus lents mais bien meilleurs.
MODEL_SIZE = "base" 

print(f"Chargement du modèle Whisper ({MODEL_SIZE})...")
try:
    model = whisper.load_model(MODEL_SIZE)
    print("Modèle chargé.")

    start_time = time.time()
    print(f"Démarrage de la transcription de {AUDIO_FILE}...")
    
    # Transcrit l'audio
    # "language='fr'" peut aider si vous savez que c'est uniquement du français
    result = model.transcribe(AUDIO_FILE, language="fr", fp16=False)
    
    end_time = time.time()
    print(f"Transcription terminée en {end_time - start_time:.2f} secondes.")
    
    # Sauvegarde le texte dans un fichier
    with open(TEXT_OUTPUT, "w", encoding="utf-8") as f:
        f.write(result["text"])
        
    print(f"Dialogues sauvegardés dans : {TEXT_OUTPUT}")

    # (Optionnel) Pour avoir les timestamps (horodatages)
    # print("\n--- Transcription avec Timestamps ---")
    # for segment in result["segments"]:
    #     start = segment['start']
    #     end = segment['end']
    #     text = segment['text']
    #     print(f"[{start:.2f}s -> {end:.2f}s] {text}")

except Exception as e:
    print(f"Une erreur est survenue : {e}")
    print("Assurez-vous que FFMPEG est installé et que le fichier audio existe.")